###### Visão Geral do Projeto

Este projeto foi pensado como um projeto de estudo, um MVP (Produto Mínimo Viável) de como deveria ser um projeto real envolvendo dados e aprendizado de máquina. O conjunto de dados tem algumas lacunas, pois não é abundante e carece de variáveis diversas para tornar o dataframe mais denso.

Enquanto estudava como configurar aprendizado de máquina para este projeto, percebi que poderia ser mais complicado do que o esperado. Com o conjunto de dados atual, os modelos só conseguem fornecer previsões "rasas". Tive que fazer uma escolha: ou continuar com o MVP como está, ou enriquecer o dataset adicionando notas de vizinhança baseadas em pontos de interesse próximos, como hospitais, supermercados, shoppings, parques e praias (como deveria ser).

Apesar de tentador enriquecer o dataset, escolhi continuar com o MVP como está para o meu primeiro projeto. Assim, consigo concluir um projeto funcional ao mesmo tempo em que aprofundo meu entendimento de aprendizado de máquina. Pelos meus estudos iniciais sobre o que realmente é ML, percebi que usar um modelo pronto é simples, mas entender de fato como ele funciona matematicamente — e reconhecer que o pré-processamento dos dados é igualmente importante, já que afeta diretamente o desempenho do modelo — será o foco dos meus próximos projetos no GitHub.

##### **Visão Geral do Notebook**

**Neste notebook, vou mostrar o uso e a configuração de dois modelos diferentes (Regressão Linear e Árvores de Decisão) usando o Scikit-Learn, além do Matplotlib para plotar os resultados e ajudar a explicá-los.**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os 

Carregando nosso arquivo CSV em um dataframe do pandas.

In [ ]:
plt.close("all")
catalogFilePath = os.path.join("..","data", "catalog.csv")
catalog = pd.read_csv(catalogFilePath)
#Quick way to remember the content of our dataset
display(catalog[["total_monthly_value", "property_size"]].head(5))

### Modelo de Regressão Linear ::

O modelo de regressão linear usa a abordagem y = mx + c. Para uma variável dependente Y, ele encontra uma relação com uma ou mais variáveis independentes X. Isso é possível porque o modelo analisa correlações entre X e Y no conjunto de dados e, então, ajusta uma linha reta aos pontos no plano XY. Essa linha pode ser usada para prever valores futuros de Y a partir de novos valores de X.


O primeiro passo na configuração deste modelo é selecionar as variáveis Dependente e Independente, y e x, que no nosso caso serão o valor do aluguel base e os m² do imóvel.

In [ ]:
X = catalog[["total_monthly_value"]]
y = catalog[["property_size"]]
plt.scatter(X, y)
plt.xlabel("monthly rent value")
plt.ylabel("property size")

Como podemos ver, temos alguns outliers, que são pontos de dados muito discrepantes em relação aos outros na mesma faixa. Além disso, temos uma diferença de tendência base, ou seja, os valores de X e Y começam a ficar cada vez mais distantes, e ambos esses fatores não são bons para o modelo prever. Isso porque os outliers podem fazer a linha de regressão superestimar ou subestimar devido à discrepância, e a diferença de tendência base entre X e Y quebra a suposição de que os erros são distribuídos de forma uniforme. Isso torna o modelo menos confiável, já que suas previsões serão precisas apenas em um certo intervalo, mas falharão em valores muito altos ou baixos. 


Portanto, será necessário adicionar uma etapa no modelo de regressão linear: o pré-processamento. Escolhi usar o **QuantileTransformer**, pois foi o método apresentado no curso rápido de scikit-learn do FreeCodeCamp que fiz. A abordagem de quantis transforma os atributos usando informações de quantis, convertendo-os para seguir uma distribuição uniforme ou normal.
###### (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html) para mais informações.

In [ ]:
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression

x_adpt = X.values.reshape(-1, 1)
y_adpt = y.values.reshape(-1, 1)

#Scaling X  with QuantileTransformer
# n_quantiles = number of training samples (prevents warnings)
# output_distribution='uniform' -> transforms data to be between 0 and 1
qauntilet_X = QuantileTransformer(n_quantiles=len(x_adpt), output_distribution='uniform')
X_scaled = qauntilet_X.fit_transform(x_adpt)

quantilet_y = QuantileTransformer(n_quantiles=len(y_adpt), output_distribution='uniform')
y_scaled = quantilet_y.fit_transform(y_adpt)

plt.scatter(X_scaled, y_scaled)
plt.xlabel("property_size (scaled 0-1)")
plt.ylabel("total_monthly_value (scaled 0-1)")
plt.title("Treino escalado")
plt.show()



No gráfico, é visível um padrão se formando, quase intuitivamente conseguimos enxergá-lo. 

O próximo passo é configurar nossa primeira previsão.

In [ ]:
LR = LinearRegression()
LR.fit(X_scaled, y_scaled.ravel())

# --- Predict and inverse transform to original scale ---
y_pred_scaled = LR.predict(X_scaled)
y_pred = quantilet_y.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()

# --- Plot original vs predicted ---
plt.figure(figsize=(10,5))

# Original data
plt.scatter(x_adpt, y_adpt, color='blue', alpha=0.6, label='Original')

# Predicted data
plt.scatter(x_adpt, y_pred, color='red', alpha=0.6, label='Predicted')

plt.xlabel("Tamanho em m²")
plt.ylabel("Valor mensal")
plt.title("Original x Predictado após o Quantile + Linear Regression")
plt.legend()
plt.show()